# sPyTial Relationalizer Development: Extending Visualization Capabilities

**Learn how to write custom relationalizers to extend sPyTial's visualization capabilities**

This notebook demonstrates different approaches to writing relationalizers in sPyTial, allowing you to customize how your objects are visualized spatially.

## What are Relationalizers?

Relationalizers are the extensibility mechanism in sPyTial. They define how different Python objects are converted into spatial atoms and relations for visualization.

**Built-in relationalizers handle:**
- Primitives (int, str, bool, etc.)
- Collections (dict, list, set, tuple)
- Custom classes with @orientation annotations

**Custom relationalizers let you:**
- Define spatial layouts for domain-specific objects
- Extract meaningful relationships from complex data structures
- Control visualization granularity and detail

In [ ]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import diagram
from spytial import RelationalizerBase, relationalizer, Atom, Relation
# Backward compatibility imports also available:
# from spytial import DataInstanceProvider, data_provider
from typing import Any, Dict, List, Tuple
import dataclasses
from datetime import datetime, date

## Approach 1: Simple Type-Based Relationalizer

The simplest relationalizer approach handles specific types with custom visualization logic.

In [ ]:
# Example: Custom relationalizer for datetime objects
@relationalizer(priority=115)
class DateTimeRelationalizer(RelationalizerBase):
    """Custom relationalizer for datetime objects with temporal relationships."""
    
    def can_handle(self, obj: Any) -> bool:
        return isinstance(obj, (datetime, date))
    
    def relationalize(self, obj: Any, walker_func) -> Tuple[Atom, List[Relation]]:
        obj_id = walker_func._get_id(obj)
        
        if isinstance(obj, datetime):
            label = obj.strftime("%Y-%m-%d %H:%M:%S")
            atom_type = "datetime"
        else:
            label = obj.strftime("%Y-%m-%d")
            atom_type = "date"
        
        atom = Atom(
            id=obj_id,
            type=atom_type,
            label=label
        )
        
        # No relations for simple datetime objects
        return atom, []

# Test the datetime relationalizer
temporal_data = {
    'created': datetime(2024, 1, 15, 10, 30, 0),
    'updated': datetime(2024, 1, 16, 14, 45, 0),
    'launch_date': date(2024, 2, 1)
}

print("Temporal data with custom datetime relationalizer:")
diagram(temporal_data)

## Approach 2: Complex Object Decomposition

For complex objects, relationalizers can decompose them into meaningful spatial components and relationships.

In [ ]:
# Example domain: File system representation
class FileNode:
    def __init__(self, name, size=None, children=None):
        self.name = name
        self.size = size  # None for directories
        self.children = children or []  # Empty for files
    
    @property
    def is_directory(self):
        return self.size is None

@relationalizer(priority=120)
class FileSystemRelationalizer(RelationalizerBase):
    """Provider that creates hierarchical file system visualization."""
    
    def can_handle(self, obj: Any) -> bool:
        return isinstance(obj, FileNode)
    
    def relationalize(self, obj: Any, walker_func) -> Tuple[Atom, List[Relation]]:
        obj_id = walker_func._get_id(obj)
        
        if obj.is_directory:
            label = f"📁 {obj.name}/"
            atom_type = "directory"
        else:
            size_str = f" ({obj.size}B)" if obj.size else ""
            label = f"📄 {obj.name}{size_str}"
            atom_type = "file"
        
        atom = {
            "id": obj_id,
            "type": atom_type,
            "label": label
        }
        
        relations = []
        for child in obj.children:
            child_id = walker_func(child)
            relations.append(("contains", obj_id, child_id))
        
        return atom, relations

# Create a sample file system
filesystem = FileNode("project", children=[
    FileNode("src", children=[
        FileNode("main.py", size=1024),
        FileNode("utils.py", size=512)
    ]),
    FileNode("tests", children=[
        FileNode("test_main.py", size=256)
    ]),
    FileNode("README.md", size=2048)
])

print("File system with hierarchical provider:")
diagram(filesystem)

## Approach 3: Dataclass Integration

Relationalizers can integrate with Python's dataclass system to automatically extract field relationships.

In [ ]:
@dataclasses.dataclass
class Person:
    name: str
    age: int
    email: str
    manager: 'Person' = None
    direct_reports: List['Person'] = None
    
    def __post_init__(self):
        if self.direct_reports is None:
            self.direct_reports = []

@relationalizer(priority=125)
class PersonRelationalizer(RelationalizerBase):
    """Provider for Person dataclass with organizational relationships."""
    
    def can_handle(self, obj: Any) -> bool:
        return isinstance(obj, Person)
    
    def relationalize(self, obj: Any, walker_func) -> Tuple[Atom, List[Relation]]:
        obj_id = walker_func._get_id(obj)
        
        atom = {
            "id": obj_id,
            "type": "person",
            "label": f"👤 {obj.name}\n{obj.age} years\n{obj.email}"
        }
        
        relations = []
        
        # Manager relationship
        if obj.manager:
            manager_id = walker_func(obj.manager)
            relations.append(("reports_to", obj_id, manager_id))
        
        # Direct report relationships
        for report in obj.direct_reports:
            report_id = walker_func(report)
            relations.append(("manages", obj_id, report_id))
        
        return atom, relations

# Create organizational hierarchy
ceo = Person("Alice CEO", 45, "alice@company.com")
vp_eng = Person("Bob VP", 40, "bob@company.com", manager=ceo)
vp_sales = Person("Carol VP", 38, "carol@company.com", manager=ceo)
engineer = Person("Dave Dev", 28, "dave@company.com", manager=vp_eng)
salesperson = Person("Eve Sales", 32, "eve@company.com", manager=vp_sales)

# Set up the hierarchy
ceo.direct_reports = [vp_eng, vp_sales]
vp_eng.direct_reports = [engineer]
vp_sales.direct_reports = [salesperson]

org_chart = {
    'organization': ceo,
    'total_employees': 5
}

print("Organizational chart with dataclass provider:")
diagram(org_chart)

## Approach 4: Priority-Based Relationalizer Selection

Multiple relationalizers can handle the same type, with priority determining which one is used.

In [ ]:
# Example: Different views of the same data
class APIEndpoint:
    def __init__(self, path, method, handler, middleware=None):
        self.path = path
        self.method = method
        self.handler = handler
        self.middleware = middleware or []

# Detailed provider (higher priority)
@relationalizer(priority=130)
class DetailedAPIRelationalizer(RelationalizerBase):
    """Detailed view showing all endpoint components."""
    
    def can_handle(self, obj: Any) -> bool:
        return isinstance(obj, APIEndpoint)
    
    def relationalize(self, obj: Any, walker_func) -> Tuple[Atom, List[Relation]]:
        obj_id = walker_func._get_id(obj)
        
        atom = {
            "id": obj_id,
            "type": "api_endpoint",
            "label": f"🌐 {obj.method} {obj.path}"
        }
        
        relations = []
        
        # Handler relationship
        handler_id = walker_func(obj.handler)
        relations.append(("handled_by", obj_id, handler_id))
        
        # Middleware relationships
        for i, middleware in enumerate(obj.middleware):
            middleware_id = walker_func(middleware)
            relations.append((f"middleware_{i}", obj_id, middleware_id))
        
        return atom, relations

# Simple provider (lower priority - will be overridden)
@relationalizer(priority=115)
class SimpleAPIRelationalizer(RelationalizerBase):
    """Simple view showing just the endpoint info."""
    
    def can_handle(self, obj: Any) -> bool:
        return isinstance(obj, APIEndpoint)
    
    def relationalize(self, obj: Any, walker_func) -> Tuple[Atom, List[Relation]]:
        obj_id = walker_func._get_id(obj)
        
        atom = {
            "id": obj_id,
            "type": "simple_endpoint",
            "label": f"{obj.method} {obj.path}"
        }
        
        # No relations in simple view
        return atom, []

# Create API structure
api_endpoints = {
    'users_api': APIEndpoint(
        path="/api/users",
        method="GET",
        handler="get_users_handler",
        middleware=["auth_middleware", "rate_limit_middleware"]
    ),
    'create_user': APIEndpoint(
        path="/api/users",
        method="POST",
        handler="create_user_handler",
        middleware=["auth_middleware", "validation_middleware"]
    )
}

print("API endpoints with detailed provider (higher priority):")
diagram(api_endpoints)

## Approach 5: Dynamic Relationalizer Registration

Relationalizers can be registered dynamically for specific object instances.

In [ ]:
# Example: Different visualization modes for the same object
class DatabaseSchema:
    def __init__(self, name, tables):
        self.name = name
        self.tables = tables

class DatabaseTable:
    def __init__(self, name, columns, foreign_keys=None):
        self.name = name
        self.columns = columns
        self.foreign_keys = foreign_keys or []

# Schema-focused provider
class SchemaRelationalizer(RelationalizerBase):
    """Shows database schema relationships."""
    
    def can_handle(self, obj: Any) -> bool:
        return isinstance(obj, DatabaseSchema)
    
    def relationalize(self, obj: Any, walker_func) -> Tuple[Atom, List[Relation]]:
        obj_id = walker_func._get_id(obj)
        
        atom = {
            "id": obj_id,
            "type": "database_schema",
            "label": f"🗄️ {obj.name} DB\n({len(obj.tables)} tables)"
        }
        
        relations = []
        for table in obj.tables:
            table_id = walker_func(table)
            relations.append(("contains_table", obj_id, table_id))
        
        return atom, relations

# Table-focused provider
class TableRelationalizer(RelationalizerBase):
    """Shows table structure and relationships."""
    
    def can_handle(self, obj: Any) -> bool:
        return isinstance(obj, DatabaseTable)
    
    def relationalize(self, obj: Any, walker_func) -> Tuple[Atom, List[Relation]]:
        obj_id = walker_func._get_id(obj)
        
        columns_preview = ", ".join(obj.columns[:3])
        if len(obj.columns) > 3:
            columns_preview += "..."
        
        atom = {
            "id": obj_id,
            "type": "database_table",
            "label": f"📋 {obj.name}\n({len(obj.columns)} cols)\n{columns_preview}"
        }
        
        relations = []
        for fk in obj.foreign_keys:
            fk_id = walker_func(fk)
            relations.append(("references", obj_id, fk_id))
        
        return atom, relations

# Create database schema
users_table = DatabaseTable("users", ["id", "name", "email", "created_at"])
posts_table = DatabaseTable("posts", ["id", "title", "content", "user_id", "created_at"], 
                           foreign_keys=["users.id"])
comments_table = DatabaseTable("comments", ["id", "content", "post_id", "user_id"], 
                              foreign_keys=["posts.id", "users.id"])

blog_schema = DatabaseSchema("blog", [users_table, posts_table, comments_table])

# Register providers dynamically
ProviderRegistry.register_instance(blog_schema, SchemaProvider())
ProviderRegistry.register_instance(users_table, TableProvider())
ProviderRegistry.register_instance(posts_table, TableProvider())
ProviderRegistry.register_instance(comments_table, TableProvider())

print("Database schema with dynamic provider registration:")
diagram(blog_schema)

## Best Practices for Relationalizer Development

### 1. Relationalizer Priorities
- Higher priority numbers = tried first
- Use priorities to create layered specialization
- Built-in relationalizers use priorities 0-10

### 2. Relationship Design
- Choose meaningful relation names
- Consider the spatial layout implications
- Use consistent naming conventions

### 3. Performance Considerations
- `can_handle()` should be fast
- Avoid expensive computations in relationalizer logic
- Cache expensive operations when possible

### 4. Error Handling
- Relationalizers should be defensive
- Gracefully handle malformed objects
- Provide meaningful error messages

### 5. Testing
- Test with edge cases and malformed data
- Verify spatial layout makes sense
- Test relationalizer priority interactions

In [ ]:
# Example: Robust provider with error handling
@relationalizer(priority=135)
class RobustConfigRelationalizer(RelationalizerBase):
    """A robust provider example with comprehensive error handling."""
    
    def can_handle(self, obj: Any) -> bool:
        try:
            return (hasattr(obj, '__dict__') and 
                   hasattr(obj, 'name') and 
                   hasattr(obj, 'settings'))
        except Exception:
            return False
    
    def relationalize(self, obj: Any, walker_func) -> Tuple[Atom, List[Relation]]:
        try:
            obj_id = walker_func._get_id(obj)
            
            # Safe attribute access
            name = getattr(obj, 'name', 'Unknown')
            settings = getattr(obj, 'settings', {})
            
            atom = {
                "id": obj_id,
                "type": "config",
                "label": f"⚙️ {name}\n({len(settings)} settings)"
            }
            
            relations = []
            if isinstance(settings, dict):
                for key, value in settings.items():
                    try:
                        value_id = walker_func(value)
                        relations.append((str(key), obj_id, value_id))
                    except Exception as e:
                        # Log error but continue processing
                        print(f"Warning: Could not process setting '{key}': {e}")
            
            return atom, relations
            
        except Exception as e:
            # Fallback to minimal representation
            obj_id = walker_func._get_id(obj)
            atom = {
                "id": obj_id,
                "type": "error",
                "label": f"❌ Error: {str(e)[:50]}..."
            }
            return atom, []

# Test with a config-like object
class AppConfig:
    def __init__(self, name, settings):
        self.name = name
        self.settings = settings

config = AppConfig("MyApp", {
    'debug': True,
    'port': 8080,
    'database_url': 'postgresql://localhost/mydb',
    'features': ['auth', 'logging', 'metrics']
})

print("Robust config provider with error handling:")
diagram({'app_config': config})

## Summary

You've learned five key approaches to writing sPyTial relationalizers:

1. **Simple Type-Based**: Handle specific types with custom logic
2. **Complex Decomposition**: Break down complex objects into spatial components
3. **Dataclass Integration**: Leverage Python's dataclass system
4. **Priority-Based Selection**: Use priorities for layered specialization
5. **Dynamic Registration**: Register relationalizers for specific instances

**Key takeaways:**
- Relationalizers transform objects into atoms and relations
- Higher priority relationalizers are tried first
- Good relationalizers extract meaningful spatial relationships
- Error handling and robustness are crucial
- Testing with real data validates your relationalizer design

**Next steps:**
- Try writing relationalizers for your domain-specific objects
- Experiment with different relationship patterns
- Combine relationalizers with `@orientation` annotations for full control
- See **02-object-annotations.ipynb** for annotation examples